## 5.12 稠密连接网络(DenseNet)

> 1. **与ResNet的主要区别在于，DenseNet里模块B的输出不是像ResNet那样和模块A的输出相加，而是在通道维上连结**
> 2. **这样模块A的输出可以直接传入模块B后面的层。这样模块A可以和后面的层连接**

![普通网络和残差网络](img/5.12_densenet.svg)

### 5.12.1 稠密块

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch.utils as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def conv_block(in_channels, out_channels):
    blk = nn.Sequential(nn.BatchNorm2d(in_channels), nn.ReLU(), 
                        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
    return blk

In [8]:
# 每块使用相同的输出通道数
# 前向计算时,把每块的输入和输出在通道上连接
class DenseBlock(nn.Module):
    def __init__(self, num_convs, in_channels, out_channels):
        super(DenseBlock, self).__init__()
        net = []
        for i in range(num_convs):
            in_c = in_channels + i * out_channels
            net.append(conv_block(in_c, out_channels))
        self.net = nn.ModuleList(net)
        self.out_channels = in_channels + num_convs * out_channels #  计算输出通道数
    
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1) # 在通道维上连接
        return X

In [9]:
# 2个卷积块, 输入通道数为3, 输出通道为10, 最后通道数为 3 + 2 * 10 = 23
blk = DenseBlock(2, 3, 10)
X = torch.rand(4, 3, 8, 8)
Y = blk(X)
Y.shape



torch.Size([4, 23, 8, 8])

### 5.12.2 过渡层

> 1. **由于稠密块会增加通道数,所以需要过渡层控制**
> 2. **通过$1 \times 1$卷积层来减小通道数,步幅为2的平均池化层减小宽和高**

In [10]:
def transition_block(in_channels, out_channels):
    blk = nn.Sequential(nn.BatchNorm2d(in_channels), nn.ReLU(),
                       nn.Conv2d(in_channels, out_channels, kernel_size=1),
                       nn.AvgPool2d(kernel_size=2, stride=2))
    return blk

In [12]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

### 5.12.3 DenseNet模型

In [13]:
net = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [14]:
# 使用4个稠密块
# 使用过渡层来减半高和宽，并减半通道数
num_channels, growth_rate = 64, 32 # num_channels 为当前的通道数, growth_rate 为卷积层通道数
num_convs_in_dense_blocks = [4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    DB = DenseBlock(num_convs, num_channels, growth_rate)
    net.add_module('DenseBlock_%d' % i, DB)
    # 上一个稠密块的输出通道数
    num_channels = DB.out_channels
    # 在稠密块之间加入减小通道数的过渡层
    if i != len(num_convs_in_dense_blocks) - 1:
        net.add_module('transition_block_%d' % i, transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [15]:
# 最后接上全局池化层和全连接输出层
net.add_module('BN', nn.BatchNorm2d(num_channels))
net.add_module('relu', nn.ReLU())
net.add_module('global_avg_pool', d2l.GlobalAvgPool2d())  #  (Batch, num_channels, 1, 1)
net.add_module('fc', nn.Sequential(d2l.FlattenLayer(), nn.Linear(num_channels, 10)))

In [16]:
X = torch.rand((1, 1, 96, 96))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

0  output shape:	 torch.Size([1, 64, 48, 48])
1  output shape:	 torch.Size([1, 64, 48, 48])
2  output shape:	 torch.Size([1, 64, 48, 48])
3  output shape:	 torch.Size([1, 64, 24, 24])
DenseBlock_0  output shape:	 torch.Size([1, 192, 24, 24])
transition_block_0  output shape:	 torch.Size([1, 96, 12, 12])
DenseBlock_1  output shape:	 torch.Size([1, 224, 12, 12])
transition_block_1  output shape:	 torch.Size([1, 112, 6, 6])
DenseBlock_2  output shape:	 torch.Size([1, 240, 6, 6])
transition_block_2  output shape:	 torch.Size([1, 120, 3, 3])
DenseBlock_3  output shape:	 torch.Size([1, 248, 3, 3])
BN  output shape:	 torch.Size([1, 248, 3, 3])
relu  output shape:	 torch.Size([1, 248, 3, 3])
global_avg_pool  output shape:	 torch.Size([1, 248, 1, 1])
fc  output shape:	 torch.Size([1, 10])


### 5.12.4 获取数据并训练模型

In [18]:
batch_size = 256
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)